Задача 1
Нужно написать две программы:

Первая генерирует бинарный файл (min 2Гб), состоящий из случайных 32-разрядных беззнаковых целых чисел (big endian).
Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.
Реализуйте две версии:

Простое последовательное чтение
Многопоточная + memory-mapped files.
Сравните время работы.


你需要编写两个程序:
第一个生成一个由随机32位无符号整数（big endian）组成的二进制文件（最小2GB）。
第二个计算这些数字的总和（使用长算术）并找到最小和最大数字。
实现两个版本 -
1. 简单的顺序读取
2. 多线程+内存映射文件。
比较工作时间

In [4]:
import numpy as np
import time
import threading
import mmap
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [1]:
file_name = 'numbers.txt'

### create file

In [98]:
def create_file(file_name):
    numbers = np.random.randint(0,2**32-1,size=2 ** 29,dtype = np.dtype('uint32'))
    with open(file_name,'wb') as f:
        for number in numbers:
            f.write(number.tobytes())
        f.close()
create_file(file_name)

### sequential

In [5]:
def sequential(file_name):
    start_time = time.time()

    with open(file_name, 'rb') as f:
        buf = f.read()
        numbers = np.frombuffer(buf, dtype = np.dtype('uint32').newbyteorder('B'))
        
        sum_num = sum(numbers.tolist())
        max_num = numbers.max()
        min_num = numbers.min()
        
        f.close()
    result_time = time.time() - start_time
    
    print("time: ", result_time)
    print("sum: ", sum_num)
    print("max: ", max_num)
    print("min: ", min_num)

In [6]:
sequential(file_name)

time:  230.61308789253235
sum:  1152935217676459992
max:  4294967289
min:  31


### multithreaded_mmap

In [58]:
def thread_func(file_name, sum_num, min_num, max_num, thread_index, chunk_start, chunk_end):
    
    with open(file_name, 'rb') as f:
        buf = mmap.mmap(f.fileno(), length = chunk_end - chunk_start, offset = chunk_start, access = mmap.ACCESS_READ)        
        numbers = np.frombuffer(buf, dtype = np.dtype('uint32').newbyteorder('B'))
        
        sum_num[thread_index] = sum(numbers.tolist())
        max_num[thread_index] = numbers.max()
        min_num[thread_index] = numbers.min() 
        
        f.close()

In [71]:
def multithreaded_mmap(file_name, threads_num):
    start_time = time.time()
    
    threads_arr_sum = [0] * threads_num
    threads_arr_min = [2 ** 32-1] * threads_num
    threads_arr_max = [0] * threads_num
    
    threads = []
    one_chunk_size = int( 2**29 / threads_num)
    chunk_start = 0
    
    for thread_index in range(threads_num):
        chunk_end = chunk_start + one_chunk_size
        one_thread = threading.Thread(target = thread_func, args = (file_name, threads_arr_sum,threads_arr_min, 
                                                                threads_arr_max, thread_index,chunk_start, 
                                                                chunk_end))
        threads.append(one_thread)
        one_thread.start()
        chunk_start = chunk_end
           
    total_sum_num = sum(threads_arr_sum)
    total_min_num = min(threads_arr_min)
    total_max_num = max(threads_arr_max)
    
    result_time = time.time() - start_time
    
    print("time: ", result_time)
    print("sum: ", total_sum_num)
    print("min: ", total_min_num)
    print("max: ", total_max_num)

In [72]:
 multithreaded_mmap(file_name, 256)

time:  8.695041179656982
sum:  284855966453015471
min:  31
max:  4294967284
